# Azure AI Agent Service

This notebook demonstrates how to use the Azure AI Agent Service to create and interact with AI agents. Azure AI Agent Service is a managed service that helps you deploy, orchestrate, and manage AI agents that can perform tasks and engage in conversations.

In this notebook, we'll:
1. Set up the Azure AI Project Client
2. Create an AI agent with specific capabilities
3. Establish a conversation thread
4. Exchange messages with the agent
5. Process and retrieve conversation results

In [ ]:
!pip list

## Importing Required Libraries

We'll start by importing the necessary libraries to work with Azure AI Agent Service:

- `os`: For accessing environment variables
- `azure.ai.projects.AIProjectClient`: The main client for interacting with Azure AI Project services
- `azure.identity.DefaultAzureCredential`: For secure, managed identity-based authentication with Azure services
- `typing.Any`: For type annotations
- `pathlib.Path`: For handling file paths

The Azure AI Project Client allows us to create and manage agents, establish conversation threads, and handle message exchanges using Azure's secure authentication mechanisms.

In [ ]:
# Import necessary libraries for Azure AI Project Client integration
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path

## Setting Up Authentication and Configuration

Following Azure best practices for security, we'll use environment variables to store sensitive connection information rather than hardcoding credentials. This approach:

1. Uses `dotenv` to load variables from a `.env` file
2. Retrieves the connection string securely
3. Implements proper error handling if the connection string is missing
4. Uses `DefaultAzureCredential` for authentication, which:
   - Supports multiple authentication methods (managed identity, environment variables, etc.)
   - Follows the principle of least privilege
   - Enables seamless authentication across different environments

The connection string contains endpoint and authentication details needed to connect to your Azure AI Agent Service resource.

In [ ]:
# Load environment variables and configure the project client
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the environment variable
project_connection_string = os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING")

if project_connection_string is None:
    raise KeyError("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING not found in .env file")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)
print(project_connection_string)

## Creating an AI Agent

Now we'll create an AI agent with specific capabilities. In Azure AI Agent Service, an agent is defined by:

- **Model**: The underlying language model that powers the agent. We're using `gpt-4o-mini`, which offers a good balance of performance and cost-efficiency for many scenarios.
- **Name**: A unique identifier for the agent instance.
- **Instructions**: The system message that defines the agent's role, behavior, and capabilities.

The instructions "You are a travel agent that plans great vacations" establish the domain expertise of our agent. In production scenarios, you would typically provide more detailed instructions to precisely control the agent's behavior, knowledge boundaries, and response patterns.

When creating agents in production environments, consider:
- Including guardrails to prevent harmful outputs
- Providing detailed examples of expected behavior
- Implementing monitoring and logging for agent actions
- Setting appropriate rate limits and usage controls

In [ ]:
# Create an agent with specific model and instructions
agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="Agent820",
        instructions="You are a travel agent that plans great vacations",
     
    )
print(f"Created agent, agent ID: {agent.id}")


## Creating a Conversation Thread

In Azure AI Agent Service, a "thread" represents a conversation session between users and agents. Threads provide:

1. **State Management**: Maintaining context across multiple interactions
2. **Message History**: Storing the complete conversation for context and reference
3. **Persistence**: Allowing conversations to continue over time

Each thread has a unique identifier that we'll use for adding messages and instructing the agent to process them. The thread architecture enables both synchronous and asynchronous conversation patterns, making it flexible for different application scenarios.

In production applications, you might implement:
- Thread lifecycle management (archiving, deletion policies)
- Access controls for thread data
- Integration with user authentication systems

In [ ]:
# Create a thread for conversation
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

## Adding User Messages

With both our agent and thread created, we can now add user messages to start the conversation. Messages in Azure AI Agent Service have several key components:

- **Thread ID**: Links the message to a specific conversation thread
- **Role**: Identifies whether the message is from the "user" or "assistant"
- **Content**: The actual text or other content of the message

The message we're adding represents the user's initial query about planning a vacation in Seoul, South Korea. This message is stored in the thread but isn't processed immediately - this separation of message creation and processing gives you fine-grained control over the conversation flow.

Azure AI Agent Service supports multiple content types beyond just text (including images in compatible models), though this example focuses on text-based conversations.

In [ ]:
# Add a user message to the thread
message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you help me plan a sunny vacation in Seoul, South Korea?",
    )
print(f"Created message, ID: {message.id}")

## Processing the Conversation

Now we'll ask the agent to process the conversation thread and generate a response. When we call `create_and_process_run`:

1. The service creates a "run" object that represents an execution instance
2. The agent receives the conversation context from the thread
3. It processes the messages and generates an appropriate response based on its instructions
4. The response is added to the thread

A "run" in Azure AI Agent Service has different states like "queued", "in_progress", "completed", or "failed". The method waits for the run to reach a terminal state and provides status information.

This implementation includes error handling to detect and report failed runs, which is critical for production applications. In more complex scenarios, you might implement retry mechanisms for transient failures or integrate with Azure Monitor for comprehensive logging and diagnostics.

In [ ]:
# Process the thread with the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    # [END create_and_process_run]
print(f"Run finished with status: {run.status}")

if run.status == "failed":
        print(f"Run failed: {run.last_error}")

## Retrieving and Displaying Conversation Messages

The final step is to retrieve all messages from the conversation thread to see the complete exchange. This allows us to view both the original user query and the agent's detailed response.

For each message, we display:
- **Message ID**: The unique identifier for the message
- **Role**: Whether the message is from the "user" or "assistant"
- **Content**: The actual text content of the message

Azure AI Agent Service supports a rich content structure that can include different content types (text, images, etc.). In this example, we're checking the content type and displaying text content specifically.

In production applications, you might:
- Implement pagination for long conversation histories
- Filter messages based on criteria
- Transform message content for different interfaces
- Implement message retention policies

In [ ]:
# List all messages in the thread
messages = project_client.agents.list_messages(thread_id=thread.id)

for msg in messages.data:
    print(f"Message ID: {msg.id}")
    print(f"Role: {msg.role}")
    print("Content:")
    for content in msg.content:
        if content['type'] == 'text':
            print(content['text']['value'])
    print("-" * 50)

## Summary and Next Steps

In this notebook, we've demonstrated the core capabilities of Azure AI Agent Service:

1. **Setting up authentication** using Azure best practices (DefaultAzureCredential and environment variables)
2. **Creating an AI agent** with specific capabilities and instructions
3. **Establishing a conversation thread** to maintain context
4. **Adding user messages** to the conversation
5. **Processing the conversation** with the agent
6. **Retrieving and displaying** the conversation history

Azure AI Agent Service provides a managed platform for building intelligent conversational experiences without managing complex infrastructure. The service handles authentication, conversation state management, and model integration following Azure's security and compliance standards.

### Extending this Solution

You can enhance this example by:
- **Adding tools and functions** that the agent can call to perform actions
- **Implementing more complex conversation flows** with multiple messages
- **Creating specialized agents** for different domains and use cases
- **Building user interfaces** around this conversation API
- **Integrating with other Azure services** like Azure Functions or Logic Apps

### Security Considerations

When deploying to production, consider:
- Using Azure Key Vault to store connection strings and credentials
- Implementing proper RBAC (Role-Based Access Control) for your Azure AI resources
- Setting up monitoring and logging for agent activities
- Implementing content filtering and safety measures

For more information, visit the [Azure AI Agent Service documentation](https://learn.microsoft.com/en-us/azure/ai-services/agents/overview).